In [1]:
from google.colab import drive
drive.mount('/content/drive')
! pip install sentence_transformers

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 6.9 MB/s 
     |████████████████████████████████| 4.2 MB 58.3 MB/s 
     |████████████████████████████████| 1.2 MB 66.9 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 6.6 MB 60.8 MB/s 
     |████████████████████████████████| 596 kB 67.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=0c675cd6db703c96b9046dc7a53cd708be70fb1e5b7952469a8230506873005f
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import statistics
import pandas as pd
import string
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from sklearn.model_selection import train_test_split
import numpy as np
import math
from sklearn.metrics import accuracy_score
from sentence_transformers import SentenceTransformer
from sklearn.metrics import f1_score
# torch.cuda.is_available()

In [4]:

pre_path = "/content/drive/MyDrive/Web Mining/"
allmovies = pd.read_csv('/content/drive/MyDrive/Web Mining/Unused/ml-1m/movies.dat', sep="::",names=['MovieID', 'Title', 'Genres'],encoding='latin-1')
ratings = pd.read_csv('/content/drive/MyDrive/Web Mining/Unused/ml-1m/ratings.dat', sep="::", names=['UserID', 'MovieID', 'Rating', 'Timestamp'], skipinitialspace=True)
# users = pd.read_csv('data/users.dat', sep="::", names=["UserID", "Gender", "Age", "Occupation", "Zip-code"],skipinitialspace=True) 
movie_final_full_dataset = pd.read_csv(pre_path+'Final Data/movie_final_full_dataset.csv')

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


# preprocessing

In [5]:
movies=pd.merge(movie_final_full_dataset,allmovies[['MovieID']],on=["MovieID"],how='left')
ratings=pd.merge(ratings,movie_final_full_dataset[['MovieID']],on=["MovieID"],how='right')

In [6]:
import ast
def getElements(x):
    l=ast.literal_eval(x)
    s=' '
    for e in l:
        s=s+e+' '
    return s

In [7]:
movies['meta_data']=movies['Genres'].str.replace('[|]', ' ') + " " +movies['Title'].str.replace('[(\d+)]', '')+movies['director'].apply(lambda x:getElements(x))+movies['stars'].apply(lambda x:getElements(x))+movies['writer'].apply(lambda x:getElements(x) )
movies['meta_data_p']=movies['meta_data']+movies['description']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


# choose members

In [8]:
allusers= ratings['UserID']
print(len(allusers))
uniqueUsers = list(dict.fromkeys(allusers))
len(uniqueUsers)

997256


6040

In [9]:
user2movie={}
for user in uniqueUsers:
    user2movie[user]=set(ratings.loc[(ratings['UserID'] == user)]['MovieID'])

In [10]:
moviesnumbers=[]
for user in uniqueUsers:
    dic={}
    dic["UserId"]=user
    dic["No. Rated Movies"]=len(user2movie[user])
    moviesnumbers.append(dic)
dfMovieNumber=pd.DataFrame(moviesnumbers).sort_values(by=['No. Rated Movies'], ascending=False)
dfMovieNumber

,UserId,No. Rated Movies
2222,4169,2260
569,1680,1846
1462,4277,1733
667,1941,1591
400,1181,1514
...,...,...
5195,2530,19
4469,2930,19
5902,4628,19
4454,1713,18


In [13]:
user_grouped = pd.read_csv('/content/drive/MyDrive/Web Mining/Pipeline/users_grouped_by3.csv')
user_grouped

,UserID,Cluster,No. Rated Movies,Group
0,3618,0,1344,3
1,2063,0,1323,3
2,1150,0,1302,3
3,1015,0,1286,4
4,5795,0,1277,4
...,...,...,...,...
301,3892,0,477,143
302,2996,0,477,143
303,757,0,465,150
304,1842,0,465,150


In [14]:
# for _ in range(100):
#     sample_users = random.choices(uniqueUsers, k=2)
#     s= len(user2movie[sample_users[0]].intersection(user2movie[sample_users[1]]))
#     if s>0:
#         print(s)

# Content Based 

In [15]:
def get_x_y(moviesIDs,userID): # get the the movies ids and their ratings as X and y
  X=list(moviesIDs)
  y=list(ratings.loc[(ratings['MovieID'].isin(X)) & (ratings['UserID'] == userID)]['Rating'])
  return X,y
    

In [16]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def clean(text):
    return text.translate(remove_punct_dict)

In [17]:
def get_bert_embeddings(X_train,X_test,column):
    model = SentenceTransformer('bert-base-nli-mean-tokens')
    s=[]
    for element in X_train:
        s.append((movies.loc[movies['MovieID'] == element][column].iat[0]))
    s1=" ".join(s)
    sentences=[s1]
    for x in X_test:
        metaData=movies.loc[movies['MovieID'] == x][column].iat[0]
        sentences.append(clean(metaData))
    sentence_embeddings = model.encode(sentences)
    return sentence_embeddings

In [18]:

def CB_single_user_evaluate(user1,column):
  movieIds=user2movie[user1]
  X,y=get_x_y(movieIds,user1)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
  sentence_embeddings=get_bert_embeddings(X_train,X_test,column) # get the embedding
  cos=cosine_similarity([sentence_embeddings[0]],sentence_embeddings[1:]) # do the cos similarity 
  # evaluate and get the accuracy 
  y_predicted=cos[0]*5
  yt=np.where(np.array(y_test) < 4, 0, 1)
  yp=np.where(np.array(y_predicted) < 4, 0, 1)
  return accuracy_score(np.round(yt),np.round(yp)),f1_score(np.round(yt),np.round(yp))

def CB_single_user(user1,column):
  movieIds=user2movie[user1]
  X,y=get_x_y(movieIds,user1)
  allMoviesids=list(movies['MovieID'].unique())
  sentence_embeddings=get_bert_embeddings(X,allMoviesids,column) # get the embedding
  cos=cosine_similarity([sentence_embeddings[0]],sentence_embeddings[1:]) # do the cos similarity 
  # evaluate and get the accuracy 
  y_predicted=cos[0]*5
  d={}
  d["MovieID"]=allMoviesids
  d["pred"]=y_predicted
  return pd.DataFrame(d)

def CB_multiple_user(users,column):
  commonIds=user2movie[users[0]]
  for u in users:
    commonIds=commonIds.intersection(user2movie[u])
  predictions=[]
  true_values=[]
  for user in users:
    uniqueIds=user2movie[user]-commonIds
    X_test,y_test=get_x_y(commonIds,user)
    X_train,y_train=get_x_y(uniqueIds,user)
    sentence_embeddings=get_bert_embeddings(X_train,X_test,column) # get the embedding
    cos=cosine_similarity([sentence_embeddings[0]],sentence_embeddings[1:]) # do the cos similarity 
    # evaluate and get the accuracy 
    y_predicted=cos[0]*5
    predictions.append(y_predicted)
    true_values.append(y_test)
  return true_values,predictions

user1=4277
user2=1941
# c_=CB(user1,user2,'title')
# print(accuracy_score(c_[0],np.round(c_[1])))
# c_=CB(user1,user2,'description')
# print(accuracy_score(c_[0],np.round(c_[1])))
# c_=CB(user1,user2,'meta_data')
# print(accuracy_score(c_[0],np.round(c_[1])))
# c_=CB_single_user_evaluate(user1,'meta_data_p')
# print(accuracy_score(c_[0],np.round(c_[1])))
# user1_=CB_single_user_evaluate(user1,'meta_data_p')
# user1_

In [19]:
def evaluate(true_values,predictions):
# do aggregation 
# Average 
  yp=np.array(predictions[0])
  n=len(predictions)
  for l in range(1,n):
    yp=yp+np.array(predictions[l])
  yp=yp/n
  yt=np.array(true_values[0])
  n=len(true_values)
  for l in range(1,n):
    yt=yt+np.array(true_values[l])
  yt=yt/n
  # if the score is more than 3 then he likes the movie
  yt=np.where(yt < 4, 0, 1)
  yp=np.where(yp < 4, 0, 1)
  return accuracy_score(np.round(yt),np.round(yp))

# predict for all the groups we have

In [20]:
groupsID=user_grouped['Group'].unique()
results={}
group2predictions={}
for g in groupsID:
  users=list(user_grouped.loc[user_grouped['Group'] == g]['UserID'])
  # print(g,users[0],users[1])
  true_values,predictions=CB_multiple_user(users,'meta_data_p')
  group2predictions[g]=predictions
  results[g]=evaluate(true_values,predictions)
# path="/content/drive/MyDrive/Web Mining/Pipline/cb_predictions.p"
# pickle.dump( group2predictions, open( path, "wb" ) )
statistics.mean(list(results.values()))

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

0.5739723019558503

Accuracy: 57.39% using avreage 

In [21]:
# statistics.mean(list(results.values()))


# How to use 

For single users update this list of user ids and it will generate a data frame with the predicted rating for the movie


In [22]:
users=dfMovieNumber['UserId'][:10]

In [23]:
# CB_single_user_evaluate(user2,'meta_data_p')

In [24]:
results_acc=[]
results_f1=[]
users=dfMovieNumber['UserId'][:100]
for i in users:
  acc,f1=CB_single_user_evaluate(i,'meta_data_p')
  results_acc.append(acc)
  results_f1.append(f1)
statistics.mean(results_acc)
statistics.mean(results_f1)

0.06783532605196427

In [ ]:
dfMovieNumber

,UserId,No. Rated Movies
2222,4169,2260
569,1680,1846
1462,4277,1733
667,1941,1591
400,1181,1514
...,...,...
5195,2530,19
4469,2930,19
5902,4628,19
4454,1713,18


In [ ]:
results_acc

[0.4778761061946903,
 0.43031123139377536,
 0.20893371757925072,
 0.6012558869701727,
 0.7838283828382838,
 0.729235880398671,
 0.8205607476635514,
 0.7102272727272727,
 0.8188824662813102,
 0.38791423001949316]